# PGURE-SVT HyperSpy Demonstration

### Tom Furnival ([tjof2@cam.ac.uk](mailto:tjof2@cam.ac.uk))

PGURE-SVT is an algorithm designed to denoise image sequences acquired in microscopy. It exploits the correlations between consecutive frames to form low-rank matrices, which are then recovered using a technique known as nuclear norm minimization. An unbiased risk estimator for mixed Poisson-Gaussian noise is used to automate the selection of the regularization parameter, while robust noise and motion estimation maintain broad applicability to many different types of microscopy.

This example notebook shows how PGURE-SVT can be combined with [HyperSpy](http://hyperspy.org), which is an open-source Python library that makes signal handling and processing straightforward in Python, with a friendly API. While you can use `pguresvt.pguresvt.SVT` to denoise a numpy array directly, `pguresvt.hspysvt.HSPYSVT` can directly denoise a HyperSpy signal.

In [ ]:
# Configures the plotting backend
#%matplotlib inline
%matplotlib qt

In [ ]:
import numpy as np
# Import the HyperSpy API
import hyperspy.api as hs

In [ ]:
# Import the HyperSpy wrapper for PGURE-SVT
from pguresvt import hspysvt

## 1. Simulated dataset

First, we load the simulated dataset using HyperSpy.

In [ ]:
# Load example dataset
movie = hs.load("examplesequence.tif")

# Truncate to 25 frames, and plot the result
movie = movie.inav[:25]
movie.plot(navigator='slider')

Now we corrupt it with a mixture of Poisson and Gaussian noise.

In [ ]:
# First extract the data and rescale to [0,1] range
clean = movie._data_aligned_with_axes
clean = clean / np.amax(clean)

# Detector gain
gain = 0.1
# Detector offset
offset = 0.1
# Detector variance
sigma = 0.1

def addnoise(x):
    return gain * np.random.poisson(x / gain) + offset + sigma * np.random.randn()
addnoise = np.vectorize(addnoise, otypes=[np.float])

noisy = addnoise(clean)

noisy_movie = hs.signals.Image(noisy)
noisy_movie.plot()

Next we initialise the SVT denoising function. The full list of options (with default values) is:

```
hspysvt.HSPYSVT(patchsize=4,
                patchoverlap=1,
                length=15,
                optimize=True,
                threshold=0.5,
                estimatenoise=True,
                alpha=-1., 
                mu=-1., 
                sigma=-1.,                
                arpssize=7, 
                tol=1e-7,
                median=5,
                hotpixelthreshold=10)
```

In this example we do not use the noise estimation procedure, and instead provide the known parameters to the algorithm directly. This information is used by the PGURE optimizer to calculate the threshold.

In [ ]:
# Initialize with suggested parameters
svt = hspysvt.HSPYSVT(patchsize=4,
                      estimatenoise=False,
                      alpha=gain,
                      mu=offset,
                      sigma=sigma,
                      tol=1e-6)

Now we are able to run the denoising and plot the result:

In [ ]:
# Run the denoising
denoised_movie = svt.denoise(noisy_movie)

# Plot denoised data
denoised_movie.plot()

### 2. Time-resolved ADF-STEM image sequence

In this example we apply PGURE-SVT to an experimental dataset of a nanoparticle acquired using ADF-STEM. Here the noise levels are not known, so a noise estimation procedure is used.

In [ ]:
# Load example dataset
expt_movie = hs.load("experimentalsequence.tif")

# Initialize with suggested parameters
expt_svt = hspysvt.HSPYSVT(patchsize=4,
                           patchoverlap=2,                     
                           tol=1e-6)

# Run the denoising
denoised_movie = expt_svt.denoise(expt_movie)

# Plot denoised data
denoised_movie.plot()